#Actividad 6. Ejemplo de solución

### Etapa 1

*Implementa un generador para los conjuntos de datos de entrenamiento, validación y prueba. En caso de ser necesario, aplica el aumento de datos para generar más imágenes.
Antes de iniciar la implementación determina ¿cuál debería ser la forma de los datos de entrada y salida esperada para tu modelo?*



Para este ejemplo de solución, implementaremos un **generador de datos que reciba como entrada una lista de cadenas de texto** con la siguiente información:

**"images_path/image_name.jpg,x_min,y_min,x_max,y_max"**

Inicialmente el archivo **airplanes.csv** contiene las anotaciones en el siguiente formato.

**"image_name.jpg,x_min,y_min,x_max,y_max"**

Por lo que sería necesario agregar a cada anotación la ruta en donde se encuentran las imágenes.

Opcionalmente, para reducir el trabajo del generador de datos durante la etapa de entrenamiento, podemos escalar el *boundingbox (x_min, y_min, x_max, y_max)* de cada imagen acorde a sus dimensiones: (height, width, 3). Esto es:

*  x_min = x_min/width 
*  y_min = y_min/height
*  x_max = x_max/width
*  x_min = x_min/height

Para implementar la escala, necesitariamos abrir cada imagen para consultar los valores de: width y height.

Nota: considere el manejo de los tipos de datos.

La **salida del generador** un **dataset**, en donde cada elemento es una dupla: (x, y). Tal que

*  x:  imágen de dimensiones (h, w, 3)
*  y:  boundingbox (x_min, y_min, x_max, y_max) 

In [22]:
import cv2 as cv
import random
import tensorflow as tf
from tensorflow.data import AUTOTUNE

In [1]:
img_path= "C:\Users\nuno\Desktop\deep-learning-data\activity6\dataset\images"
annotations_path = "C:\Users\nuno\Desktop\deep-learning-data\activity6\dataset\airplanes.csv"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2550021102.py, line 1)

In [6]:
def preprocess_annotation(img_src, annotation):
  filename, x_min, y_min, x_max, y_max = annotation.split(",")
  img_path = img_src + "/" + filename
  img = cv.imread(img_path)
  h, w, _ = img.shape
  annotation = "".join([img_path, ",", str(float(x_min)/w), ",",
                        str(float(y_min)/h), ",",
                        str(float(x_max)/w), ",",
                        str(float(y_max)/h)])
  return annotation

In [7]:
txt = preprocess_annotation(img_path, "image_0002.jpg,59,35,342,153")
print(txt)

/content/drive/MyDrive/DL2023p-licd/S06/Actividad 6/data/images/image_0002.jpg,0.14713216957605985,0.19021739130434784,0.8528678304239401,0.8315217391304348


In [9]:
def build_datasets(images_path, annotation_path, train_split=0.95, val_split=0.1):
  
  annotations = open(annotation_path).read().splitlines()
  examples = [ preprocess_annotation(images_path, item) for item in annotations ]
  
  random.shuffle(examples)

  #Aplicar el split al conjunto de datos
  s = int(len(examples)*train_split)
  train_examples = examples[:s]
  test_examples = examples[s:]

  s = int(len(train_examples)*val_split)
  val_examples = train_examples[:s]
  train_examples = train_examples[s:]

  return train_examples, val_examples, test_examples

In [10]:
train_data, val_data, test_data = build_datasets(img_path, annotations_path, 0.95, 0.1)

In [13]:
print(train_data[0])

/content/drive/MyDrive/DL2023p-licd/S06/Actividad 6/data/images/image_0485.jpg,0.14285714285714285,0.16560509554140126,0.8771929824561403,0.8280254777070064


In [21]:
print(len(val_data))

76


In [18]:
def loadExample(example):
  # Extraer de la cadena image, bbox 
  str_tensors = tf.strings.split(example, sep=",")

  # Cargar la imagen
  img = tf.io.read_file(str_tensors[0])
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.convert_image_dtype(img, dtype=tf.float16)
  img = tf.image.resize(img, (128, 128))

  x_min = tf.strings.to_number(str_tensors[1])
  y_min = tf.strings.to_number(str_tensors[2])
  x_max = tf.strings.to_number(str_tensors[3])
  y_max = tf.strings.to_number(str_tensors[4])

  bbox = [x_min, y_min, x_max, y_max]
          
  return img, bbox

In [19]:
img, bbox = loadExample(train_data[0])

In [20]:
print(bbox)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.14285715>, <tf.Tensor: shape=(), dtype=float32, numpy=0.1656051>, <tf.Tensor: shape=(), dtype=float32, numpy=0.877193>, <tf.Tensor: shape=(), dtype=float32, numpy=0.82802546>]


In [23]:
# Pipelines 

batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices(train_data)
train_dataset = (train_dataset
                 .shuffle(len(train_data))
                 .map(loadExample, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .batch(batch_size)
                 .prefetch(AUTOTUNE)
                 )

val_dataset = tf.data.Dataset.from_tensor_slices(val_data)
val_dataset = (val_dataset
                 .shuffle(len(val_data))
                 .map(loadExample, num_parallel_calls=AUTOTUNE)
                 .cache()
                 .batch(batch_size)
                 .prefetch(AUTOTUNE)
                 )

### Etapa 2
Para la arquitectura de tu modelo, se sugiere que elijas como base uno de los modelos preentrenados de tensorflow.keras.applications, por ejemplo: resnet, xception o vgg.



### Etapa 3

En caso de ser necesario, agrega capas de entrada, capas ocultas y de salida para que tu modelo realiza de forma apropiada la tarea de localización. Para determinar la configuración de la capa de salida, recuerda cuál es la salida esperada para tu modelo.

### Etapa 4

Entrena y valida tu modelo.

### Etapa 5

Evalua tu modelo utilizando las imágenes de tu conjunto de pruebas.